In [4]:
import tensorflow_datasets as tf
from tensorflow.keras.utils import to_categorical

In [6]:
(train_ds, train_labels), (test_ds, test_labels) = tf.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]